In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import sys
import cv2
import numpy as np

import simple_csi_camera as csi

print(sys.version)

3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]


In [3]:
# check to see if camera is available
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 Nov 10 23:59 /dev/video0


In [4]:
def laplacian_edge(img):
    """
    return laplacian image
    """
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian

In [5]:
def canny_edge(img):
    """
    return canny edge image
    """
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 100, 150)
    return canny

In [6]:
def detect_face(img):
    """
    return face detect image with Haar-cascade
    """
    
    # read default haarcascade features
    face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('./data/haarcascade_eye.xml')
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # face detection
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        detected_face = cv2.rectangle(img, (x, y),(x+w, y+h), (255, 0, 0), 2)
        
        # region of interest
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = detected_face[y:y+h, x:x+w]
        
        # eyes detection
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
        
#         return detected_face

In [7]:
def show_converted_camera(face_detection=False):
    print(csi.gstreamer_pipeline(flip_method=0))
    
    # load from CSI Camera(raspi camera v2)
    cap = cv2.VideoCapture(csi.gstreamer_pipeline(flip_method=0),
                           cv2.CAP_GSTREAMER)
    
    # check camera normally
    if cap.isOpened():
        window_handle = cv2.namedWindow('CSI Camera', cv2.WINDOW_AUTOSIZE)  # window setting
        
        # when window closed: return -1
        while cv2.getWindowProperty('CSI Camera', 0) >= 0:
            ret_val, frame = cap.read()
            
            # processing
            if face_detection:
                detect_face(frame)
                window_face = cv2.namedWindow('CSI Camera - Face detection', cv2.WINDOW_AUTOSIZE)
                cv2.imshow('CSI Camera - Face detection', frame)
            
            img = canny_edge(frame)
            cv2.imshow('CSI Camera', img)
            
            key_code = cv2.waitKey(30) & 0xFF
            
            # Stop program with ESC key
            if key_code == 27:
                break
        
        # Freeing camera object
        cap.release()
        cv2.destroyAllWindows()
    
    else:
        print('Unable to open camera')

In [8]:
show_converted_camera(face_detection=True)

nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, format=(string)NV12, framerate=(fraction)60/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink
